## Load embedding model

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings

EMBEDDING_MODEL_NAME = "sdadas/mmlw-retrieval-roberta-large"

embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

In [2]:
from qdrant_client import QdrantClient

collection_name = f"poquad_{EMBEDDING_MODEL_NAME.replace('/', '_')}"
client = QdrantClient(host='localhost', port=6333)

In [3]:
from langchain.vectorstores import Qdrant

qdrant = Qdrant(
    client=client,
    collection_name=collection_name,
    embeddings=embeddings
)

In [4]:
retriever = qdrant.as_retriever()

In [5]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_name = './output/roberta-base-squad2-pl/checkpoint-8500'

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
from transformers import pipeline

nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [9]:
def get_context(query, k):
    results = retriever.get_relevant_documents(
        k=k,
        query=query,
        fetch_k=k*5,
    )

    context = ""

    for result in results:
        context += result.page_content + " "
        

    return context

In [15]:
question = "Gdzie rozegrany zostanie pojedynek o Superpuchar?"
context = get_context(question, 1)

print(context)

result = nlp(question=question, context=context)

print(result)

Od 2014 roku w Superpucharze, z inicjatywy prezesa PZPN Zbigniewa Bońka, nastąpiła zmiana, gdyż po 8 latach przerwy związek podjął na swoje barki organizację rozgrywek o Superpuchar. O trofeum walczyć będą Mistrz Polski oraz zdobywca Pucharu Polski sezonu zakończonego w roku rozgrywania Superpucharu. Tym samym powrócono do nazwy Superpuchar Polski. Mecz rozgrywany będzie na stadionie Mistrza Polski, na około tydzień przed startem nowego sezonu Ekstraklasy. W sytuacji, gdy ten sam klub sięgnie po mistrzostwo i Puchar Polski, jego rywalem w meczu o trofeum będzie finalista ostatniej edycji Pucharu Polski. Od 2014 roku w Superpucharze, z inicjatywy prezesa PZPN Zbigniewa Bońka, nastąpiła zmiana, gdyż po 8 latach przerwy związek podjął na swoje barki organizację rozgrywek o Superpuchar. O trofeum walczyć będą Mistrz Polski oraz zdobywca Pucharu Polski sezonu zakończonego w roku rozgrywania Superpucharu. Tym samym powrócono do nazwy Superpuchar Polski. Mecz rozgrywany będzie na stadionie Mi

## Evaluation

In [16]:
from datasets import load_dataset

poquad = load_dataset("clarin-pl/poquad")

In [17]:
poquad_validation = poquad["validation"]

In [18]:
import evaluate

metric = evaluate.load("squad")

In [ ]:
from transformers import pipeline

def get_predictions(eval_dataset, qa_pipeline):
    predictions = []
    for question, id in zip(eval_dataset["question"], eval_dataset["context"], eval_dataset["id"]):
        context = get_context(question, 5)
        answer = qa_pipeline(question=question, context=context)
        prediction = {
            'id': id,
            'prediction_text': answer['answer']
        }

        predictions.append(prediction)

    return predictions